Lindbald Master Equation and Hamiltonian

---

$$ \dot\rho=-i[H_{1},\rho]+\gamma_{1D}[2\sigma_{1}\rho\sigma_{1}^{\dagger}-\sigma_{1}^{\dagger}\sigma_{1}\rho-\rho\sigma_{1}^{\dagger}\sigma_{1}] $$

$$ H_{1}=[\omega_{0}+A_{j}(t)]\sigma_{1}^{\dagger}\sigma_{1}-\frac{i\Omega_{R}}{2}(e^{-i\varepsilon t}\sigma_{1}^{\dagger}-H.c) $$

Hamiltonian with Detuning



---
$$ U(t)=e^{i\epsilon t\sigma_{1}^{\dagger}\sigma_{1}} $$

$$ H_{1}^{'}=U^{\dagger}H_{1}U - iU^{\dagger}\frac{\partial U}{\partial t} $$

$$ iU^{\dagger}\frac{\partial U}{\partial t} = i(-i\epsilon)\sigma_{1}^{\dagger}\sigma_{1} = -\epsilon\sigma_{1}^{\dagger}\sigma_{1} $$

$$ U^{\dagger}(e^{-i\epsilon t}\sigma_{1}^{\dagger})U = e^{-i\epsilon t} \cdotp e^{i\epsilon t} \sigma_{1}^{\dagger} = \sigma_{1}^{\dagger}$$

$$ \delta = \omega_{0} - \epsilon $$

$$ H^{'}_{1}(t) = [\delta + A_{j}(t)]\sigma_{1}^{\dagger}\sigma_{1} - \frac{i\Omega_{R}}{2}(\sigma_{1}^{\dagger} - \sigma_{1}) $$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import expm

%matplotlib inline

In [2]:
gamma_1D = 1.0   # Decay rate
D = 0.5          # Detuning d = (omega_0 - epsilon)
Omega = 5        # Modulation frequency
Omega_R = 0.05   # Rabi frequency
A = 0.025        # Modulation amplitude

In [13]:
def pauli_matrices():
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

    # Lowering operator (sigma_-)
    sigma = np.array([[0, 1], [0, 0]], dtype=complex)

    # Raising operator (sigma_+)
    sigma_dag = np.array([[0, 0], [1, 0]], dtype=complex)

    # Number operator
    n = sigma_dag @ sigma

    # Identity
    I = np.eye(2, dtype=complex)

    return sigma_x, sigma_y, sigma_z, sigma, sigma_dag, n, I

sigma_x, sigma_y, sigma_z, sigma, sigma_dag, n, I = pauli_matrices()

def A_modulation(t):
    return A * np.cos(Omega * t)

def hamiltonian(t):
    A_t = A_modulation(t)
    return (D + A_t) * n - ((1j * Omega_R) / 2) * (sigma_dag - sigma)

def commutator(A, B):
    return (A @ B) - (B @ A)

def lindblad_term(rho, s):
    s_dag = s.conj().T
    return 2 * s @ rho @ s_dag - s_dag @ s @ rho - rho @ s_dag @ s

def lindblad_master_equation(t, rho_vec):
    rho = rho_vec.reshape((2, 2))
    H = hamiltonian(t)  # Hamiltonian at time (t)
    drho_dt = (-1j * commutator(H, rho)) + (gamma_1D * lindblad_term(rho, sigma))
    return drho_dt.flatten()

In [22]:
psi0 = np.array([1, 0], dtype=complex)  # Initial ground state, outer product
rho0 = np.outer(psi0, psi0.conj())
rho0_vec = rho0.flatten()

t_final = 10 / gamma_1D
times = np.linspace(0, t_final, 1000)

print("[*] Starting integration.")
solution = solve_ivp(
    lindblad_master_equation,
    t_span=(0, t_final),
    y0=rho0_vec,
    t_eval=times,
    method='DOP853',  # RK45
)

print("[*] Integration complete!")

# Observables from density matrix
n_expect = np.zeros(len(solution.t))
sigma_dag_expect = np.zeros(len(solution.t), dtype=complex)
sigma_expect = np.zeros(len(solution.t), dtype=complex)

for i, t in enumerate(solution.t):
    rho = solution.y[:, i].reshape((2, 2))

    # Expectation values
    n_expect[i] = np.trace(rho @ n).real
    sigma_dag_expect[i] = np.trace(rho @ sigma_dag)
    sigma_expect[i] = np.trace(rho @ sigma)

    print(sigma_expect)

Streaming output truncated to the last 5000 lines.
[ 0.        +0.00000000e+00j -0.000249  +6.53171744e-07j
 -0.00049551+2.59508095e-06j -0.00073956+5.79924509e-06j
 -0.00098115+1.02390962e-05j -0.00122031+1.58879889e-05j
 -0.00145706+2.27192212e-05j -0.00169141+3.07060531e-05j
 -0.00192338+3.98217250e-05j -0.00215299+5.00394764e-05j
 -0.00238026+6.13325672e-05j -0.0026052 +7.36742992e-05j
 -0.00282783+8.70380390e-05j -0.00304818+1.01397242e-04j
 -0.00326625+1.16725477e-04j -0.00348207+1.32996451e-04j
 -0.00369565+1.50184035e-04j -0.00390702+1.68262291e-04j
 -0.00411618+1.87205498e-04j -0.00432317+2.06988177e-04j
 -0.00452799+2.27585119e-04j -0.00473066+2.48971411e-04j
 -0.00493121+2.71122461e-04j -0.00512965+2.94014026e-04j
 -0.00532599+3.17622232e-04j -0.00552026+3.41923604e-04j
 -0.00571248+3.66895084e-04j -0.00590265+3.92514057e-04j
 -0.00609081+4.18758372e-04j -0.00627696+4.45606359e-04j
 -0.00646112+4.73036849e-04j -0.00664332+5.01029192e-04j
 -0.00682356+5.29551832e-04j -0.00700